# Extract and display an adjacency matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 08.2025

## Summary
This analysis extracts and visualizes the connectivity between all pairs of pre- and post-synaptic neurons (adjacency matrix), optionally including the number of synapses per connection (synaptome matrix).
For details, see the [README](README.md).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import adjacency
from datetime import datetime
from entitysdk import Client, ProjectContext, models
from ipywidgets import widgets, interact
from obi_auth import get_token
from obi_notebook import get_projects
from obi_notebook import get_entities


## Circuit selection and download

A SONATA circuit unser `./analysis_circuit/circuit_config.json` will be analyzed. To place the data there, we must select 
the circuit, then download it. 

Should the circuit of interest already be placed at that location, you can skip ahead to the section `Circuit analysis` below.

#### Project selection
As a first step we select one of the projects we have access to that the circuit is associated with. If the circuit of interest is part of the public OBI assets, any project can be selected.

In [ ]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

#### Circuit selection

Next, we select the circuit. If you already know the unique identifier of the circuit of interest, paste it below into line 4 of the next cell.

Otherwise, a widget for circuit selection will be created that allows you to simply mark the circuit of interest.

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

# Optional: Download using unique ID
entity_ID = "<CIRCUIT-ID>"  # <<< FILL IN UNIQUE CIRCUIT ID HERE


if entity_ID != "<CIRCUIT-ID>":
    circuit_ids = [entity_ID]
else:
# Alternative: Select from a table of entities
    circuit_ids = []
    circuit_ids = get_entities.get_entities("circuit", token, circuit_ids,
                                            project_context=project_context,
                                            multi_select=False, exclude_scales=["single"],
                                            show_pages=True, page_size=12,
                                            default_scale="small", add_columns=["subject.name"])

#### Fetch circuit
The circuit is copied to the local system at the expected location.

In [ ]:
# Fetch circuit
fetched = client.get_entity(entity_id=circuit_ids[0], entity_type=models.Circuit)
print(f"Circuit fetched: {fetched.name} (ID {fetched.id})\n")
print(f"#Neurons: {fetched.number_neurons}, #Synapses: {fetched.number_synapses}, #Connections: {fetched.number_connections}\n")
print(f"{fetched.description}\n")

# Download SONATA circuit files
asset = [asset for asset in fetched.assets if asset.label=="sonata_circuit"][0]
asset_dir = asset.path 
circuit_dir = "analysis_circuit_" + datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')

t0 = time.time()
client.download_directory(
    entity_id=fetched.id,
    entity_type=models.Circuit,
    asset_id=asset.id,
    output_path=circuit_dir,
    max_concurrent=4,  # Parallel file download
)
t = time.time() - t0
print(f"Circuit files downloaded to '{os.path.join(circuit_dir, asset_dir)}' in {t:.1f}s")

## Circuit analysis

By default, the circuit from the download location is used. If a circuit from some other location us used, please modify the `circuit_config = ...` path below accordingly.

In [ ]:
# Path to existing circuit config
circuit_config = os.path.join(circuit_dir, asset_dir, "circuit_config.json")

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [ ]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

## Adjacency matrix extraction

Extract the adjacency/synaptome matrix using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [ ]:
adj_syn_dict = adjacency.compute(c, sel_src=pre_nset_wdgt.value, sel_dest=post_nset_wdgt.value, edges_popul_name=e_popul_wdgt.value)

## Interactive visualization

Interactive visualization of adjacency or synaptome matrix. Marker scaling and alpha (transparency) can be adjusted for better display of small/large matrices.

In [ ]:
# Interactive plot function
def plot_fct(res_sel, mscale, alpha):
    res_dict = adj_syn_dict[res_sel]
    cmap = "hot_r"

    mat = res_dict["data"].tocoo()  # Convert to COO, for easy access to row/col and data!!
    col_idx = mat.data
    vmin = 0
    vmax = max([1, *col_idx])
    ms = mscale * 5000 / max(mat.shape),  # Adjust marker size for proper display

    plt.figure()
    plt.scatter(mat.col, mat.row, marker=",", s=ms, edgecolors="none", alpha=alpha, c=col_idx, cmap=cmap, vmin=vmin, vmax=vmax, label="Conn")

    plt.xlabel("Post-neurons" + ("" if post_nset_wdgt.value is None else f" [{post_nset_wdgt.value}]"))
    plt.ylabel("Pre-neurons" + ("" if pre_nset_wdgt.value is None else f" [{pre_nset_wdgt.value}]"))
    plt.title(res_sel_wdgt.label, fontweight="bold")

    plt.axis("image")
    plt.xlim((-0.5, res_dict["data"].shape[1] - 0.5))
    plt.ylim((-0.5, res_dict["data"].shape[0] - 0.5))
    plt.gca().invert_yaxis()
    if res_dict["data"].dtype != bool:
        cb = plt.colorbar(label="#Synapses")
        cb_ticks = np.unique(np.round(cb.get_ticks()))
        cb.set_ticks(cb_ticks[(cb_ticks >= vmin) & (cb_ticks <= vmax)])
    plt.tight_layout()
    plt.show()

In [ ]:
res_sel_wdgt = widgets.Dropdown(options=[("Adjacency matrix", "adj"), ("Synaptome matrix", "adj_cnt")], description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
mscale_wdgt = widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=1, step=0.01, description="Marker scale:", style={"description_width": "auto"})
alpha_wdgt = widgets.FloatSlider(value=1.0, min=0.01, max=1.0, step=0.01, description="Marker alpha:", style={"description_width": "auto"})
iplot = interact(plot_fct, res_sel=res_sel_wdgt, mscale=mscale_wdgt, alpha=alpha_wdgt)